In [13]:
import plotly
import pandas as pd
import numpy as np

plotly.offline.init_notebook_mode(connected=True)

In [3]:
df = pd.read_csv('city_info_geocodio.csv')
df.head()

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City.1,State.1,County,Zip,Country,Source,Congressional District
0,Aurora,Illinois,41.747452,-88.238622,1.0,place,NaN,NaN,Aurora,IL,DuPage County,60504.0,US,TIGER/Line® dataset from the US Census Bureau,IL11
1,Carbondale,Illinois,37.739004,-89.209109,1.0,place,NaN,NaN,Carbondale,IL,Jackson County,62901.0,US,TIGER/Line® dataset from the US Census Bureau,IL12
2,Champaign,Illinois,40.107890,-88.244266,1.0,place,NaN,NaN,Champaign,IL,Champaign County,61820.0,US,TIGER/Line® dataset from the US Census Bureau,IL13
3,Chicago,Illinois,41.885310,-87.622130,1.0,place,NaN,NaN,Chicago,IL,Cook County,60601.0,US,TIGER/Line® dataset from the US Census Bureau,IL7
4,Cicero,Illinois,41.838036,-87.759870,1.0,place,NaN,NaN,Cicero,IL,Cook County,60804.0,US,TIGER/Line® dataset from the US Census Bureau,IL4


In [52]:
# Geocodio told me that five values weren't succesfful mapped, but for the sake of best practice, I checked anyway
# I suspect that these either weren't named correctly or for some reason weren't in geocodio's database 
# Because there are only five missing values, I'll fix this manually, although that obviously isn't a scalable 
# solution. 

df.loc[df['Latitude'] == 0]

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City.1,State.1,County,Zip,Country,Source,Congressional District


Geocodio tells you the likely accuracy of the lat/long it's provided with an accuracy score. Some of the scores do not have high accuracy scores, which is worth mentioning. For now, we'll see if any mapping errors actually occur. If they do, I'll go back and engineer a solution.

In [51]:
#Updating relevent values 

new_data = [(40.0874759, -108.8048292, 'CO3'), (39.446649, -106.03757, 'CO2'), (28.022243, -81.732857, 'FL9'),
            (39.9689532, -82.9376804, 'OH3'), (39.103119, -84.512016, 'OH1')]
indexes = (63, 85, 98, 186, 205)

for i in range(5): 
    df.loc[indexes[i],'Latitude'] = new_data[i][0]
    df.loc[indexes[i], 'Longitude'] = new_data[i][1]
    df.loc[indexes[i], 'Congressional District'] = new_data[i][2]

In [55]:
# This creates the text that appears at each point on the the scroll over
df['text'] = df['City'] + ', ' + df['State'] + '.' + ' Status: Active'

In [77]:
#Building the actual map here 


# If at a later point we choose to add a "success/failure scale" or something similar, we'll add it using this 

scl = [ [0,"rgb(5, 10, 172)"],[1,"rgb(220, 220, 220)"] ]
# green - 127, 255, 0   and   red  - 

data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df['Longitude'],
        lat = df['Latitude'],
        hovertext = df['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = 'darkred',
#             cmax = df['cnt'].max(),
#             colorbar=dict(
#                 title="Incoming flightsFebruary 2011" (need to add another paren. before end bracket to readd)
            )
        )]

layout = dict(
        title = 'Targeted Cities with Active Anti-Panhandling Laws',
        colorbar = False,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(156, 156, 156)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )

# This code is for use in displaying the map in Juypter notebook. 

# plotly.offline.iplot(fig, validate=False, filename='city_locations')

# This code is used to create standalone HTML that is saved locally and opened in the browser.

plotly.offline.plot(fig, validate=False, filename='city_locations.html')

'file:///Users/flatironschool/Documents/NLCHP_Map/city_locations.html'